### Example of TimeMCL for Solar dataset

This notebook contains an example for training TimeMCL on the Solar dataset.

### Setup

First create the conda virtual environment with the required packages with `cd tsExperiments ; bash setup-env.sh`. Then activate it before running the next cells.

### Training

Training can be performed with the following commands

In [ ]:
import os
import sys
import numpy as np
import rootutils
import torch 
rootutils.setup_root(search_from='.', indicator=".project-root", pythonpath=True)

sys.path.append(os.path.dirname(os.environ["PROJECT_ROOT"]))
sys.path.append(os.path.join(os.environ["PROJECT_ROOT"], "tsExperiments"))
from tsExperiments.scripts_plot.plottimeMCL import plot_mcl
from gluonts.dataset.repository import get_dataset
from tsExperiments.models.project_models.tMCL.personnalized_evaluator import (
    MultivariateEvaluator,
)
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from tsExperiments.models.project_models.tMCL.personnalized_evaluator import MultivariateEvaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from tsExperiments.models.project_models.tMCL.timeMCL_estimator import timeMCL_estimator

In [ ]:
dataset_name = "solar_nips" 

dataset = get_dataset(dataset_name, regenerate=False)
metadata = dataset.metadata

datasets_params = {
    "exchange_rate_nips": {"num_feat_dynamic_real": 4},
    "solar_nips": {"num_feat_dynamic_real": 4},
    "electricity_nips": {"num_feat_dynamic_real": 4},
    "traffic_nips": {"num_feat_dynamic_real": 4},
    "taxi_30min": {"num_feat_dynamic_real": 6},
    "wiki-rolling_nips": {"num_feat_dynamic_real": 2},
}

n_hyp = 8
fast_dev_run = True
device = "cpu" if not torch.cuda.is_available() else "gpu"
max_target_dim = min(2000, int(metadata.feat_static_cat[0].cardinality))
train_grouper = MultivariateGrouper(max_target_dim=max_target_dim)
test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                max_target_dim=max_target_dim)
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

# timeMCL configuration

model_params = {
  'beta': 1.0,
  'num_hypotheses': 4,
  'mcl_hidden_dim': 300,
  'num_layers': 2,
  'num_cells': 40,
  'cell_type': "LSTM",
  'num_parallel_samples': 100,
  'dropout_rate': 0.1,
  'embedding_dimension': 0,
  'conditioning_length': 100,
  'loss_type': "l2",
  'residual_layers': 8,
  'residual_channels': 8,
  'dilation_cycle_length': 2,
  'scaling': True,
  'pick_incomplete': False,
  'time_features': None,
  'mcl_loss_type': "min_ext_sum", 
  'num_feat_dynamic_real': datasets_params[dataset_name]['num_feat_dynamic_real'],
  'score_loss_weight': 0.5,
  'wta_mode': "relaxed-wta",
  'wta_mode_params': {
    'epsilon': 0.1,
    'temperature_ini': 10,
    'temperature': 10,
    'scheduler_mode': "exponential",
    'temperature_decay': 0.95,
    'temperature_lim': 5e-4,
    'wta_after_temperature_lim': True},
  'optim_kwargs':{
    'lr': 1e-3,
    'weight_decay': 1e-8,
    'patience': 10,
  },
  'sample_hyps': True,
  'single_linear_layer': True,
  'backbone_deleted': True,
  'scaler_type': "mean",
  'div_by_std': False,
  'minimum_std': 1e-3,
  'minimum_std_cst': 1e-4,
  'default_scale': False,
  'default_scale_cst': False,
  'add_minimum_std': False,
  }

trainer_kwargs = {
    'max_epochs': 5 if fast_dev_run else 200,
    'accelerator': device,
    'gradient_clip_val': 10.0,
    'validation_only': False,
}

data_kwargs = {
    'num_batches_per_epoch': 30,
    'num_batches_val_per_epoch': 100,
    'batch_size': 200,
    'shuffle_buffer_length': None,
    'train': {
    'type': "Gluonts_ds",
    'split_train_val': True,
    'n_pred_steps_val': 10, # number of step of validation (as a factor of the prediction length)
    'dataset_name': dataset_name,
    'num_feat_dynamic_real': datasets_params[dataset_name]['num_feat_dynamic_real']},
    'eval' : {
    'type': "Gluonts_ds",
    'dataset_name': "solar_nips",
    'num_feat_dynamic_real': datasets_params[dataset_name]['num_feat_dynamic_real']},
    }

estimator = timeMCL_estimator(
            freq=metadata.freq,
            prediction_length=metadata.prediction_length,
            target_dim=max_target_dim,
            context_length=metadata.prediction_length,
            trainer_kwargs=trainer_kwargs,
            data_kwargs=data_kwargs,
            **model_params,
        )

predictor = estimator.train(dataset_train)

### Inference and evaluation

Please run the following commands for inference and evaluation

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset_test, 
    predictor=predictor, 
    num_samples=n_hyp
)
forecasts = list(forecast_it)
targets = list(ts_it)

evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                target_agg_funcs={'sum': np.sum})

agg_metric, _, distorsion = evaluator(
                targets, forecasts, num_series=len(dataset_test)
            )
            
agg_metric["Distorsion"] = distorsion

print("RMSE: {}".format(agg_metric["m_sum_RMSE"]))
print("Distorsion: {}".format(agg_metric["Distorsion"]))
print("CRPS-Sum: {}".format(agg_metric["m_sum_mean_wQuantileLoss"]))

### Plotting

In [ ]:
figure_path = "tmcl_example_{}.png".format(dataset_name)

# Formatting first window forecasts & observations
hypothesis_forecasts = forecasts[0].samples  # shape (N, forecast_length, target_dim)
target_df = targets[0]

# plot tMCL forecasts
plot_mcl(
        target_df=target_df,
        hypothesis_forecasts=hypothesis_forecasts,
        forecast_length=metadata.prediction_length,
        context_points=metadata.prediction_length,
        rows=3,
        cols=2,
        plot_mean=True,
        freq_type=metadata.freq,
        fname=figure_path,
        is_mcl=True,  
        extract_unique=True  
)

print('Figure saved in {}'.format(figure_path))